# Trigram-Based Model for Text Analysis and Generation

This Jupyter Notebook demonstrates the process of building and utilizing a trigram-based model for text analysis and generation. The workflow includes the following steps:

- **Cleaning and Preprocessing of Text Data from Project Gutenberg**:
    - Load and clean text data from Project Gutenberg to prepare it for analysis.
    - Remove unwanted characters and normalize the text.

- **Building a Trigram Model for Patterns in English**:
    - Construct a trigram model to capture patterns of three consecutive characters in the cleaned text.
    - Count the occurrences of each trigram to understand the frequency of character sequences.

- **Generating Text Using the Trigram Model**:
    - Use the trigram model to generate new text that mimics the style and structure of the original text.
    - Implement a function to produce a specified length of text based on the trigram probabilities.

- **Analyzing the Validity of Generated Text**:
    - Evaluate the generated text by calculating the percentage of valid English words.
    - Compare the generated text to a dictionary of valid words to assess its coherence.

- **Exporting the Trigram Model in JSON Format**:
    - Save the trigram model to a JSON file for future use and sharing.
    - Ensure the model is easily accessible and reusable.

This notebook provides a comprehensive guide to creating and utilizing a trigram-based model for text analysis and generation, offering insights into the patterns and structures of English text.


### Task 1: Third-order Letter Approximation Model

In [1]:
import re
from collections import defaultdict
import random
import string
import json
import unittest
from unittest.mock import patch
from io import StringIO

##### `clean_text` function
Cleans the input text by performing the following operations:
1. Replaces newline characters with spaces.
2. Removes non-alphabetic characters, keeping only uppercase letters, spaces, and full stops.
3. Replaces multiple consecutive spaces with a single space.


In [2]:
## Step 1: Load and Clean the Text
def clean_text(text):
    """
    Cleans the input text by performing the following operations:
    1. Replaces newline characters with spaces.
    2. Removes non-alphabetic characters, keeping only uppercase letters, spaces, and full stops.
    3. Replaces multiple spaces with a single space.
    Args:
        text (str): The input text to be cleaned.
    Returns:
        str: The cleaned text.
    """
    # Replace newlines with spaces.
    text = text.replace('\n', ' ')
                        
    # Remove non-alphabetic characters. Keep letters, spaces and full stops
    cleaned = re.sub(r'[^A-Z\s.]','', text.upper())

    # Replace multiple spaces with a single space
    cleaned = re.sub(r'\s+',' ',cleaned).strip()

    return cleaned

##### `process_file` Function

The `process_file` function is designed to load and clean the text from a specified file. Below is an example of how to use this function on a single file:

1. **Load and Clean the Text**:
    - The function reads the content of the file located at the specified path.
    - It searches for specific start and end markers to extract the main content.
    - The extracted text is then cleaned using the `clean_text` function.

2. **Example Usage**:
    - In this example, the text is loaded and cleaned from the file `frankenstein.txt` located in the `gutenbergtexts` directory.
    - The first 500 characters of the cleaned text are displayed.



In [3]:
# Function to load in Text files and clean them

def process_file(file_path):
    """
    Processes a text file by extracting and cleaning its content.

    This function reads the content of a text file located at the specified 
    'file_path'. It searches for the standard Project Gutenberg start and end 
    markers to extract the main content of the text. If the markers are found, 
    the text between them is extracted. If the markers are not found, a warning 
    message is printed. The extracted text is then cleaned and returned.

    Args:
        file_path (str): The path to the text file to be processed.

    Returns:    
        str: The cleaned text extracted from the file.
    """
    # Open the file located at 'file_path' in read mode with utf-8 encoding
    with open(file_path, 'r', encoding='utf-8') as f:
        # Read the entire content of the file into the variable 'text'
        text = f.read()

    # Search for the start marker indicating the beginning
    start_marker = re.search(r"\*\*\* START OF (THE|THIS) PROJECT GUTENBERG EBOOK.*\*\*\*", text)
    # Search for the end marker indicating the end of the content
    end_marker = re.search(r"\*\*\* END OF (THE|THIS) PROJECT GUTENBERG EBOOK.*\*\*\*", text)

    # If both the start and end markers are found, extract the text between them
    if start_marker and end_marker:
        text = text[start_marker.end():end_marker.start()]
    else:
        # If markers are not found, print a warning message
        print("Warning: Could not find standard Project Gutenberg markers.")

    # Clean the text
    return clean_text(text)

#####  Example of how to use the function on a single file:

* **Example Usage**:
    - In this example, the trigram model is built from the cleaned text of `frankenstein.txt`.
    - The first 10 trigram counts from the model are displayed.

In [4]:
# ********** Example of how to use the function on a single file:  *********************

# Load and clean the text from a file (in this case, 'Frankenstein')
# cleaned_text = process_file('gutenbergtexts/frankenstein.txt')

# Display the first 500 characters of Frankenstein
# print(cleaned_text[:500])  

##### `build_trigram_model` Function

The `build_trigram_model` function constructs a trigram model from the given cleaned text. A trigram model is a dictionary where the keys are trigrams (3-character sequences) and the values are the counts of how often each trigram appears in the text.

#### Explanation
1. **Initialization**: A dictionary `trigram_counts` is initialized to count the occurrences of each trigram.
2. **Trigram Creation**: The function loops through the text to create trigrams. A trigram consists of 3 consecutive characters, so the loop stops 2 characters before the end to avoid index out-of-range errors.
3. **Count Incrementation**: For each trigram, the count in the dictionary is incremented.
4. **Return**: The dictionary of trigram counts is returned.


In [5]:
# Function to build a trigram model from the cleaned text
def build_trigram_model(cleaned_text):
    """
    Builds a trigram model from the given cleaned text.

    A trigram model is a dictionary where the keys are trigrams (3-character sequences)
    and the values are the counts of how often each trigram appears in the text.

    Args:
        cleaned_text (str): The input text from which to build the trigram model. 
                            It should be preprocessed and cleaned.

    Returns:
        defaultdict: A dictionary with trigrams as keys and their occurrence counts as values.
    """
    # Initialize a dictionary to count the occurrences of each trigram
    trigram_counts = defaultdict(int)

    # Loop through the text and the create trigrams
    # A trigram consists of 3 consecutive characters, so we iterate over the text, 
    # stopping 2 characters before the end to avoid index out-of-range errors
    for i in range(len(cleaned_text) -2):
        # Extract the current trigram (3-character sequence)
        trigram = cleaned_text[i:i+3]
        # Increment the count of this trigram in the dictionary
        trigram_counts[trigram] += 1

    # Return the dictionary of trigram counts
    return trigram_counts

##### `process_multiple_files` function

Description
This function takes a list of file paths, processes each file to clean the text, builds a trigram model for each file, and then combines the trigram counts from all files into a single dictionary. The combined trigram counts are returned as the output.

1. **Clean the Text**: It processes each file to clean the text, which typically involves removing unwanted characters, converting text to lowercase, etc.
2. **Build Trigram Model**: For each cleaned text, it constructs a trigram model. A trigram is a sequence of three consecutive words, and the model counts the occurrences of each trigram.
3. **Combine Trigram Counts**: It then combines the trigram counts from all the processed files into a single dictionary. This dictionary accumulates the counts of each trigram across all files.


In [6]:
# Function to process multiple text files and build a combined trigram model
def process_multiple_files(file_paths):
    """
    Processes multiple text files to build a combined trigram model.
    This function takes a list of file paths, processes each file to clean the text,
    builds a trigram model for each file, and then combines the trigram counts from
    all files into a single dictionary.
    Args:
        file_paths (list of str): A list of file paths to the text files to be processed.
    Returns:
        defaultdict: A dictionary containing the combined trigram counts from all files,
                     where the keys are trigrams (tuples of three words) and the values
                     are the counts of those trigrams across all files.
    """
    # Initialize a dictionary to store trigram counts across all files
    combined_trigram_counts = defaultdict(int)

    # Loop through the list of file paths
    for file_path in file_paths:

        # Process/Clean the file
        cleaned_text = process_file(file_path)

        # Build trigram model for the current file
        trigram_counts = build_trigram_model(cleaned_text)


        # Merge the trigram counts from this file into the combined count
        for trigram, count in trigram_counts.items():
            combined_trigram_counts[trigram] += count
        
    # Return the combined trigram counts from all files
    return combined_trigram_counts


List all file paths for 5 different books from Project Gutenberg and process them to build a combined trigram model.

The function performs the following steps:
1. Lists the file paths for five books from Project Gutenberg.
2. Processes all the files and builds a combined trigram model from the listed file paths.
3. Displays the first 10 trigram counts from the combined trigram model.

File paths:
- 'gutenbergtexts/frankenstein.txt'
- 'gutenbergtexts/mobydick.txt'
- 'gutenbergtexts/prideandprejudice.txt'
- 'gutenbergtexts/romeoandjuliet.txt'
- 'gutenbergtexts/scarletletter.txt'

The combined trigram model is created by calling the `process_multiple_files` function with the list of file paths.
The first 10 trigram counts from the combined trigram model are displayed by converting the model to a list of tuples and printing the first 10 items.



In [7]:
# List all file paths for 5 different books from Project Gutenberg
file_paths = [
    'gutenbergtexts/frankenstein.txt',
    'gutenbergtexts/mobydick.txt',
    'gutenbergtexts/prideandprejudice.txt',
    'gutenbergtexts/romeoandjuliet.txt',
    'gutenbergtexts/scarletletter.txt'
]

# Process all the files and build a combined trigram model from the listed file paths
combined_trigram_model = process_multiple_files(file_paths)

# Display the first 10 trigram counts from the combined trigram model
print(dict(list(combined_trigram_model.items())[:100])) # Convert to a list of tuples and display the first 10 

{'LET': 1288, 'ETT': 997, 'TTE': 2141, 'TER': 7254, 'ER ': 17193, 'R T': 4524, ' TO': 16087, 'TO ': 14617, 'O M': 1842, ' MR': 1372, 'MRS': 374, 'RS.': 716, 'S. ': 3141, '. S': 1466, ' SA': 3993, 'SAV': 180, 'AVI': 512, 'VIL': 479, 'ILL': 3706, 'LLE': 1195, 'LE ': 6435, 'E E': 2250, ' EN': 2286, 'ENG': 723, 'NGL': 984, 'GLA': 350, 'LAN': 1307, 'AND': 19336, 'ND.': 311, 'D. ': 1902, ' ST': 5071, 'ST.': 309, 'T. ': 2434, '. P': 340, ' PE': 2722, 'PET': 180, 'ETE': 587, 'ERS': 3578, 'RSB': 3, 'SBU': 18, 'BUR': 315, 'URG': 149, 'RGH': 64, 'GH ': 1784, 'H D': 294, ' DE': 4535, 'DEC': 579, 'EC.': 3, 'C. ': 70, '. T': 3299, ' TH': 55430, 'TH ': 7714, 'H .': 11, ' . ': 311, '. Y': 568, ' YO': 5124, 'YOU': 5050, 'OU ': 3929, 'U W': 495, ' WI': 8644, 'WIL': 1842, 'LL ': 7835, 'L R': 313, ' RE': 6192, 'REJ': 103, 'EJO': 52, 'JOI': 192, 'OIC': 276, 'ICE': 1039, 'CE ': 4594, 'E T': 10499, 'O H': 1994, ' HE': 13123, 'HEA': 2504, 'EAR': 4471, 'AR ': 2179, 'THA': 8516, 'HAT': 9320, 'AT ': 12970, 'T N'

### Task 2: Third-order letter approximation generation

##### `print_trigram_possibilities` function

Steps
1. Filter Trigrams:

2. The function searches the trigram_model dictionary for trigrams that start with the given pair of characters (last_two).
These matching trigrams and their counts are stored in the possible_trigrams dictionary.
Extract and Count:

3. The function extracts the third character (the next possible character) from each matching trigram.
Counts the occurrences of each matching trigram.
Calculate Probabilities:

4. Computes the probability of each next character by dividing the count of each trigram by the total count of all matching trigrams.
Print Results:

5. For each next character, the function prints the following:
The trigram (formed by combining last_two and the next character).
The count of the trigram.
The probability of the trigram.
Handle Missing Data:

6. If no trigrams start with last_two, the function prints a message and exits.

In [8]:
def print_trigram_possibilities(trigram_model, last_two):
    """
    Prints the possible next characters for a given pair of characters (last_two)
    and their probabilities based on the trigram model.
    
    Args:
        trigram_model (dict): The trigram model containing counts of trigrams.
        last_two (str): The last two characters (e.g., 'TH') for which to calculate next character probabilities.
    """

    # Find all trigrams with the given two characters
    possible_trigrams = {trigram: count for trigram, count in trigram_model.items() if trigram.startswith(last_two)}

    if not possible_trigrams:
        print(f"No trigrams found starting with '{last_two}'.")
        return
    
    # Seperate the third letter and their respective counts
    letters = [trigram[2] for trigram in possible_trigrams.keys() ]
    counts = list(possible_trigrams.values())

    # Calculate the total count of occurrence's for normalisation
    total_count = sum(counts)

    # Print the possibilities
    print(f"Possible next characters after '{last_two}':")
    for letter, count in zip(letters, counts):
        probability = count / total_count
        print(f"{last_two + letter}: appeared {count} times, probability = {probability:.4f}")
    print(f"Total occurrences; {total_count}\n")



##### `generate_text` function

The generated text was analyzed to determine the percentage of valid English words. The analysis involved the following steps:

1. **Loading Valid Words**:
    - A set of valid English words was loaded from the `words.txt` file.

2. **Generating Text**:
    - Text was generated using the trigram model built from the combined text of five books from Project Gutenberg.

3. **Calculating Word Percentage**:
    - The generated text was processed to extract individual words.
    - Each word was checked against the set of valid English words.
    - The percentage of valid English words in the generated text was calculated.


In [9]:

def generate_text(trigram_model, length=10000, line_length=80):
    """
    Generates a text string based on a trigram model.
    Args:
        trigram_model (dict): A dictionary where keys are trigrams (strings of three characters)
                              and values are their respective counts or probabilities.
        length (int, optional): The desired length of the generated text. Default is 10,000 characters.
        line_length (int, optional): The length of each line in the formatted output text. Default is 80 characters.
    Returns:
        str: A generated text string based on the trigram model, formatted with line breaks at the specified line length.
    Notes:
        - The function starts generating text with the string "TH".
        - It continues generating characters based on the trigram model until the desired length is reached.
        - If no trigrams are found for the last two characters in the generated text, the generation process stops.
        - The generated text is formatted with line breaks at the specified line length.
    """
    # Start with the string "TH" 
    generated_text = "TH"

    # Continue generating characters until reached desired length
    while len(generated_text) < length:
        # Get the last two characters from the current text
        last_two = generated_text[-2:]

        # Find all trigrams starting with those two characters
        possible_trigrams = {trigram: count for trigram, count in trigram_model.items() if trigram.startswith(last_two)}

        if not possible_trigrams:
            # In case there are no trigrams starting with the last two characters, stop generating
            print(f"Warning: No trigrams found for the pair '{last_two}'.")
            break

        # Separate the third letter and their respective counts
        letters = [trigram[2] for trigram in possible_trigrams.keys()]
        counts = list(possible_trigrams.values())

        next_char = random.choices(letters, weights=counts, k=1)[0]

        generated_text += next_char
    # Add line breaks at the specified line length (default 80 characters per line)
    formatted_text = '\n'.join([generated_text[i:i+line_length] for i in range(0, len(generated_text), line_length)])

    return formatted_text

# Generate the text and print possible trigrams for debugging (set debug=True)
generated_text = generate_text(combined_trigram_model, length=10000, line_length=80)


# Display the first 500 characters of the generated text
print(f"\nGenerated text (first 500 characters):\n{generated_text[:500]}")
print("\n\n")
print_trigram_possibilities(combined_trigram_model, "TH")


Generated text (first 500 characters):
THE BY PROME MATHER AN INFRIESS THE PERS WAS CAUT RUFFER THE FERCLIS THE TEATED 
LADARE SUPONLY WITHYST. AND STECT REACK OVIS LIFIR. WITS WAS AYSEETHIS PLIGHBOAR
BETTLE THE BE LIM AITHE THE WOR DOSSIBEW SMAIN OF YOUT APPON UPOSIMED REMED THIS
HE CH A CON EXAM OFBAT SHOSTON ALETTED THATILD ME AMP BE HORSE WAS GLET HADY AD 
BALLAM THATURINTATE SO LEN WHING TURNMIS BY HADDED SLOVELL WARK AN HIMINE BOU CA
ND BE WAS STERED BLE IT REVOULD SMAD TOU HOW ON ANOW LING FRIBLAYSE SHAS OCCULOO
 MANDUCCULTER 



Possible next characters after 'TH':
TH : appeared 7714 times, probability = 0.1080
THA: appeared 8516 times, probability = 0.1192
THE: appeared 42957 times, probability = 0.6013
THI: appeared 5546 times, probability = 0.0776
THO: appeared 3509 times, probability = 0.0491
THS: appeared 291 times, probability = 0.0041
THU: appeared 437 times, probability = 0.0061
THR: appeared 1257 times, probability = 0.0176
TH.: appeared 290 times, probability = 0.004

### Task 3. Analyze your model

##### `load_words` Function

The `load_words` function is designed to load a set of valid English words from a specified file. It reads each line from the file, strips any whitespace (including newline characters) from the ends, and converts each word to lowercase to ensure uniformity. The function returns a set of these valid words, which can be used for various text analysis tasks, such as validating the words in generated text.

In [10]:
def load_words(file_path):
    """
    Load a set of valid words from a file.

    This function reads a file located at the specified file path, where each line
    in the file represents a word. It strips any whitespace from the ends of each
    line and converts each word to lowercase to ensure uniformity. The words are
    stored in a set to eliminate duplicates and allow for efficient membership testing.

    Args:
        file_path (str): The path to the file containing the words.

    Returns:
        set: A set of valid words loaded from the file, all in lowercase.
    """
    ''''''
     # Open the file located at 'file_path' in read mode
    with open(file_path, 'r') as f:
        # Use a set comprehension to read each line from the file,
        # strip any whitespace (including newline characters) from the ends,
        # and convert each word to lowercase to ensure uniformity.
        valid_words = {line.strip().lower() for line in f}
    # Return the set of valid words loaded from the file
    return valid_words

##### `extract_words` function
This function removes punctuation, converts the text to lowercase, and splits it into individual words. 
It is used to preprocess text for further analysis, ensuring consistency in word extraction.


In [11]:
def extract_words(text):
    """
    Extracts words from a given text by splitting on non-alphabetic characters.
    
    Args:
        text (str): The input text to extract words from.
        
    Returns:
        list: A list of extracted words from the text.
    """

    # Remove punctuation and convert to lowercase
    translator = str.maketrans('','',string.punctuation)
    clean_text = text.translate(translator).lower()

    # Split by spaces to get words
    words = clean_text.split()
    return words

##### `calculate_word_percentage` Function

The `calculate_word_percentage` function is designed to evaluate the coherence of the generated text by calculating the percentage of valid English words it contains. This function performs the following steps:

1. **Extract Words from Generated Text**:
    - The function uses the `extract_words` function to preprocess the generated text by removing punctuation, converting it to lowercase, and splitting it into individual words.

2. **Count Valid Words**:
    - Each extracted word is checked against a set of valid English words (`valid_words`).
    - The function counts how many of the extracted words are valid English words.

3. **Calculate Percentage**:
    - The percentage of valid English words is calculated by dividing the count of valid words by the total number of extracted words.
    - The function returns the percentage of valid words, the count of valid words, and the total number of words.

This function helps in assessing the quality of the generated text by providing a quantitative measure of its validity based on the presence of recognizable English words.

In [12]:
def calculate_word_percentage(generated_text, valid_words):
    """
    Calculates the percentage of valid English words in the generated text.
    
    Args:
        generated_text (str): The generated text from Task 2.
        valid_words (set): A set of valid English words.
        
    Returns:
        float: The percentage of valid words in the text.
    """

    # Extract words from the generated text
    words = extract_words(generated_text)

    # Count the valid words
    valid_word_count = sum(1 for word in words if word in valid_words)

    # Calculate the percentage of valid words there
    total_words = len(words)
    percentage = (valid_word_count / total_words) * 100 if total_words > 0 else 0

    return percentage, valid_word_count, total_words

##### `load_words` function

The function performs the following steps:

1. **Load Valid English Words**:
    - Loads a list of valid English words from `data/words.txt` using the `load_words` function.

2. **Generate Text**:
    - Utilizes the text generated in Task 2 by calling the `generate_text` function with the combined trigram model and specifying a length of 10,000 characters.

3. **Calculate Percentage of Valid English Words**:
    - Calculates the percentage of valid English words in the generated text using the `calculate_word_percentage` function.

4. **Print Results**:
    - Prints the total number of valid English words, the total number of words generated, and the percentage of valid words.

In [13]:
# Load the valid english words from words.txt
valid_words = load_words('data/words.txt')

# Use the generated text from Task 2
generated_text = generate_text(combined_trigram_model, length=10000)

# Calculate the percentage of valid English words
percentage, valid_word_count, total_words = calculate_word_percentage(generated_text, valid_words)

# Print the results
print(f"Valid words: {valid_word_count} / {total_words}")
print(f"Percentage of valid English words: {percentage:.2f}%")

Valid words: 689 / 1881
Percentage of valid English words: 36.63%


### Task 4: Export your model as JSON

In this task, we will export the combined trigram model to a JSON file. The combined trigram model is stored in the variable `combined_trigram_model`, which is a `defaultdict` containing the counts of trigrams. The output file will be named `trigrams.json`.

The following steps will be performed:
1. Define the `export_trigram_model` function to export the trigram model to a JSON file.
2. Specify the name of the output file.
3. Call the function to export the trigram model.
4. Print a confirmation message indicating that the trigram model has been exported.

The `export_trigram_model` function is defined as follows:


In [14]:
def export_trigram_model(trigram_model, output_file):
    """
    Exports the trigram model to a JSON file.
    
    Args:
        trigram_model (dict): The trigram model containing counts of trigrams.
        output_file (str): The path to the output JSON file.
    """

    # Open the specified output file in write mode
    with open(output_file, 'w') as f:
        # Use json.dump() to write the trigram model to the file
        json.dump(trigram_model, f, indent=4) # indent = 4, this is for nice printing

In [15]:
# Specify the name of the output file where the trigram model will be saved
output_file = 'trigrams.json'
# Call the function to export the trigram model to a JSON file
# 'combined_trigram_model' is the dictionary containing the trigrams and their counts
export_trigram_model(combined_trigram_model,output_file)
# Print the results
print(f"Trigram model has been exported to {output_file}")

Trigram model has been exported to trigrams.json


### Conclusion

In this Jupyter Notebook, we successfully implemented a trigram-based model for text analysis and generation. The workflow included the following tasks:

1. **Cleaning and Preprocessing of Text Data**:
    - Loaded and cleaned text data from Project Gutenberg, removing unwanted characters and normalizing the text to prepare it for analysis.

2. **Building a Trigram Model**:
    - Constructed a trigram model to capture patterns of three consecutive characters in the cleaned text.
    - Counted the occurrences of each trigram to understand the frequency of character sequences.

3. **Generating Text Using the Trigram Model**:
    - Used the trigram model to generate new text that mimics the style and structure of the original text.
    - Implemented a function to produce a specified length of text based on the trigram probabilities.

4. **Analyzing the Validity of Generated Text**:
    - Evaluated the generated text by calculating the percentage of valid English words.
    - Compared the generated text to a dictionary of valid words to assess its coherence.
    - Generated text contained approximately 37% valid English words.

5. **Exporting the Trigram Model in JSON Format**:
    - Saved the trigram model to a JSON file for future use and sharing.
    - This ensures that the model is easily accessible and reusable.

Overall, this notebook provided a comprehensive guide to creating and utilizing a trigram-based model for text analysis and generation, offering insights into the patterns and structures of English text. The generated text, while not perfect, demonstrated the potential of trigram models in capturing and replicating language patterns.

##### Tests for Task 1
Using Python's `unittest` library, we will test the functions implemented in Task 1.

In [16]:
class TestCleanText(unittest.TestCase):
    def test_clean_text(self):
        self.assertEqual(clean_text("Hello, World!"), "HELLO WORLD")
        self.assertEqual(clean_text("This   is\na test."), "THIS IS A TEST.")
        self.assertEqual(clean_text("1234!@#$%^&*()"), "")
        self.assertEqual(clean_text("The end."), "THE END.")
        self.assertEqual(clean_text(""), "")


In [17]:
class TestBuildTrigramModel(unittest.TestCase):
    def test_build_trigram_model(self):
        # Test with a simple text
        text = "HELLO WORLD"
        expected_output = {
            'HEL': 1,
            'ELL': 1,
            'LLO': 1,
            'LO ': 1,
            'O W': 1,
            ' WO': 1,
            'WOR': 1,
            'ORL': 1,
            'RLD': 1
        }
        self.assertEqual(build_trigram_model(text), expected_output)

        # Test with repeated trigrams
        text = "AAA"
        expected_output = {
            'AAA': 1
        }
        self.assertEqual(build_trigram_model(text), expected_output)

        # Test with no trigrams
        text = "AB"
        expected_output = {}
        self.assertEqual(build_trigram_model(text), expected_output)

        # Test with empty string
        text = ""
        expected_output = {}
        self.assertEqual(build_trigram_model(text), expected_output)

In [18]:
import os

class TestProcessFile(unittest.TestCase):
    def test_process_file(self):
        # Create a sample text with Project Gutenberg markers
        sample_text = """
        *** START OF THIS PROJECT GUTENBERG EBOOK SAMPLE ***
        This is a sample text for testing.
        It includes multiple lines and some punctuation!
        *** END OF THIS PROJECT GUTENBERG EBOOK SAMPLE ***
        """
        # Write the sample text to a temporary file
        with open('sample.txt', 'w', encoding='utf-8') as f:
            f.write(sample_text)
        
        # Process the file
        cleaned_text = process_file('sample.txt')
        
        # Expected cleaned text
        expected_cleaned_text = "THIS IS A SAMPLE TEXT FOR TESTING. IT INCLUDES MULTIPLE LINES AND SOME PUNCTUATION"
        
        # Check if the cleaned text matches the expected output
        self.assertEqual(cleaned_text, expected_cleaned_text)
        
        # Remove the temporary file
        os.remove('sample.txt')



In [19]:
class TestProcessMultipleFiles(unittest.TestCase):
    def test_process_multiple_files(self):
        # Create mock Gutenberg files
        with open("file1.txt", "w") as file1:
            file1.write(
                "*** START OF THE PROJECT GUTENBERG EBOOK TEST ***\n"
                "HELLO WORLD\n"
                "*** END OF THE PROJECT GUTENBERG EBOOK TEST ***"
            )
        with open("file2.txt", "w") as file2:
            file2.write(
                "*** START OF THE PROJECT GUTENBERG EBOOK TEST ***\n"
                "WORLD PEACE\n"
                "*** END OF THE PROJECT GUTENBERG EBOOK TEST ***"
            )
        
        combined_model = process_multiple_files(["file1.txt", "file2.txt"])

        combined_model = dict(combined_model)

        
        expected = {
            "HEL": 1,
            "ELL": 1,
            "LLO": 1,
            "LO ": 1,
            "O W": 1,
            " WO": 1,
            "WOR": 2,
            "ORL": 2,
            "RLD": 2,
            "LD ": 1,
            "D P": 1,
            " PE": 1,
            "PEA": 1,
            "EAC": 1,
            "ACE": 1
        }
        self.assertEqual(combined_model, expected)
        os.remove("file1.txt")
        os.remove("file2.txt")

##### Tests for Task 2
Using Python's `unittest` library, we will test the functions implemented in Task 2.

In [20]:
# Mocked trigram model for testing purposes
TEST_TRIGRAM_MODEL = {
    "THA": 5, "THE": 10, "TH ": 3, "HEA": 4, "HEL": 5, "HE ": 3,
    "ELL": 6, "LO ": 4, "O W": 4, "WOR": 6, "ORL": 5, "RLD": 5,
    "LD ": 3, "D C": 3, " CE": 2, "CEA": 2, "EAC": 3, "ACE": 3,
    "E L": 4, "LLO": 3, "LO ": 2, "O T": 4, " TO": 5, "TO ": 3,
    "A T": 3, "T H": 3, "HAT": 4
}

In [21]:
class TestPrintTrigramPossibilities(unittest.TestCase):
    @patch('sys.stdout', new_callable=StringIO)
    def test_print_trigram_possibilities(self, mock_stdout):
        """
        Test that print_trigram_possibilities correctly outputs probabilities 
        based on the updated TEST_TRIGRAM_MODEL.
        """
        # Use the updated TEST_TRIGRAM_MODEL
        trigram_model = TEST_TRIGRAM_MODEL

         # Call the function
        print_trigram_possibilities(trigram_model, "TH")

        # Expected output with corrected probabilities
        expected_output = (
            "Possible next characters after 'TH':\n"
            "THA: appeared 5 times, probability = 0.2778\n"
            "THE: appeared 10 times, probability = 0.5556\n"
            "TH : appeared 3 times, probability = 0.1667\n"
            "Total occurrences; 18\n\n"
        )

        # Compare the actual output to the expected output
        self.assertEqual(mock_stdout.getvalue(), expected_output)
        
    @patch('sys.stdout', new_callable=StringIO)
    def test_print_trigram_possibilities_no_match(self, mock_stdout):
        """
        Test print_trigram_possibilities when there are no matching trigrams.
        """
        # Use the updated TEST_TRIGRAM_MODEL
        trigram_model = TEST_TRIGRAM_MODEL

        # Call the function with a non-existent pair
        print_trigram_possibilities(trigram_model, "ZZ")

        # Expected output when no matching trigrams are found
        expected_output = "No trigrams found starting with 'ZZ'.\n"

        # Compare the output
        self.assertEqual(mock_stdout.getvalue(), expected_output)


In [22]:
# Test for print_trigram_possibilities when no matching trigrams are found
class TestPrintTrigramPossibilitiesNoMatch(unittest.TestCase):
    @patch('sys.stdout', new_callable=StringIO)
    def test_print_trigram_possibilities_no_match(self, mock_stdout):
        """
        Test print_trigram_possibilities with no matching trigrams.
        """
        print_trigram_possibilities(TEST_TRIGRAM_MODEL, "ZZ")
        
        expected_output = "No trigrams found starting with 'ZZ'.\n"
        self.assertEqual(mock_stdout.getvalue(), expected_output)


In [23]:
class TestGenerateTextLength(unittest.TestCase):
    def test_generate_text_length(self):
        """
        Test that generate_text generates the correct text length.
        """
        random.seed(42)  # For deterministic output
        generated = generate_text(TEST_TRIGRAM_MODEL, length=50, line_length=20)

        # Check that the text starts with "TH"
        self.assertTrue(generated.startswith("TH"))

        # Check that the total length (ignoring line breaks) matches 50
        total_chars = len(generated.replace("\n", ""))
        self.assertEqual(total_chars, 50)


In [24]:
# Test for generate_text to ensure correct line formatting
class TestGenerateTextLineBreaks(unittest.TestCase):
    def test_generate_text_line_breaks(self):
        """
        Test that generate_text formats text with correct line breaks.
        """
        random.seed(42)  # For consistent results
        generated = generate_text(TEST_TRIGRAM_MODEL, length=100, line_length=20)

        # Split the generated text by lines
        lines = generated.split("\n")

        # Check that all lines except the last one have 20 characters
        for line in lines[:-1]:
            self.assertEqual(len(line), 20)

        # Check that the last line is not longer than 20 characters
        self.assertTrue(len(lines[-1]) <= 20)


##### Tests for Task 3
Using Python's `unittest` library, we will test the functions implemented in Task 3.

In [25]:
class TestLoadWords(unittest.TestCase):
    def test_load_words(self):
        # Create a temporary file with sample words
        sample_words = "Hello\nWorld\nTEST\nSample\n"
        with open("temp_words.txt", "w") as f:
            f.write(sample_words)

        # Run the function and compare the output
        result = load_words("temp_words.txt")
        expected = {"hello", "world", "test", "sample"}
        self.assertEqual(result, expected)

        # Clean up
        os.remove("temp_words.txt")

In [26]:
class TestExtractWords(unittest.TestCase):
    def test_extract_words(self):
        text = "Hello, World! This is a test."
        result = extract_words(text)
        expected = ["hello", "world", "this", "is", "a", "test"]
        self.assertEqual(result, expected)

In [27]:
unittest.main(argv=[''], verbosity=2, exit=False)

test_build_trigram_model (__main__.TestBuildTrigramModel.test_build_trigram_model) ... ok
test_clean_text (__main__.TestCleanText.test_clean_text) ... ok
test_extract_words (__main__.TestExtractWords.test_extract_words) ... ok
test_generate_text_length (__main__.TestGenerateTextLength.test_generate_text_length)
Test that generate_text generates the correct text length. ... ok
test_generate_text_line_breaks (__main__.TestGenerateTextLineBreaks.test_generate_text_line_breaks)
Test that generate_text formats text with correct line breaks. ... ok
test_load_words (__main__.TestLoadWords.test_load_words) ... ok
test_print_trigram_possibilities (__main__.TestPrintTrigramPossibilities.test_print_trigram_possibilities)
Test that print_trigram_possibilities correctly outputs probabilities ... ok
test_print_trigram_possibilities_no_match (__main__.TestPrintTrigramPossibilities.test_print_trigram_possibilities_no_match)
Test print_trigram_possibilities when there are no matching trigrams. ... ok
t